In [20]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import *

In [3]:
#Start session Context com nome ENEM
spark = SparkSession.builder.appName("COVID").getOrCreate()

In [52]:
%%time
covid = (
    spark
    .read
    .format("csv")
    .options(header=True, sep=",")
    .load("WHO-COVID-19-global-data.csv")
 )


Wall time: 166 ms


In [87]:
 covid = covid.withColumn('Date_reported', to_date(unix_timestamp(col('Date_reported'), 'yyyy-MM-dd').cast("timestamp")))
 covid = covid.withColumn("New_cases",covid.New_cases.cast('integer')) 
 covid = covid.withColumn("Cumulative_cases",covid.Cumulative_cases.cast('integer')) 
 covid = covid.withColumn("New_deaths",covid.New_deaths.cast('integer')) 
 covid = covid.withColumn("Cumulative_deaths",covid.Cumulative_deaths.cast('integer')) 

In [88]:
covid.printSchema()

root
 |-- Date_reported: date (nullable = true)
 |-- Country_code: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- WHO_region: string (nullable = true)
 |-- New_cases: integer (nullable = true)
 |-- Cumulative_cases: integer (nullable = true)
 |-- New_deaths: integer (nullable = true)
 |-- Cumulative_deaths: integer (nullable = true)



In [82]:
%%time
#quantidade de registros
covid.count()

Wall time: 179 ms


200006

In [83]:
covid.show()

+-------------+------------+-----------+----------+---------+----------------+----------+-----------------+
|Date_reported|Country_code|    Country|WHO_region|New_cases|Cumulative_cases|New_deaths|Cumulative_deaths|
+-------------+------------+-----------+----------+---------+----------------+----------+-----------------+
|   2020-01-03|          AF|Afghanistan|      EMRO|        0|               0|         0|                0|
|   2020-01-04|          AF|Afghanistan|      EMRO|        0|               0|         0|                0|
|   2020-01-05|          AF|Afghanistan|      EMRO|        0|               0|         0|                0|
|   2020-01-06|          AF|Afghanistan|      EMRO|        0|               0|         0|                0|
|   2020-01-07|          AF|Afghanistan|      EMRO|        0|               0|         0|                0|
|   2020-01-08|          AF|Afghanistan|      EMRO|        0|               0|         0|                0|
|   2020-01-09|          AF|

In [106]:
%%time
# primeiro caso no brasil
consulta1 = (covid.filter((covid.Country == 'Brazil')&(covid.New_cases >0)).sort(col("Date_reported").asc()) )
consulta1.show(1)

+-------------+------------+-------+----------+---------+----------------+----------+-----------------+
|Date_reported|Country_code|Country|WHO_region|New_cases|Cumulative_cases|New_deaths|Cumulative_deaths|
+-------------+------------+-------+----------+---------+----------------+----------+-----------------+
|   2020-02-27|          BR| Brazil|      AMRO|        1|               1|         0|                0|
+-------------+------------+-------+----------+---------+----------------+----------+-----------------+
only showing top 1 row

Wall time: 338 ms


In [108]:
%%time
#Qual foi o dia em que o Brasil reportou o maior número de casos?
consulta2 = (covid.filter((covid.Country == 'Brazil')&(covid.New_cases >0)).sort(col("New_cases").desc()) )
consulta2.show(1)

+-------------+------------+-------+----------+---------+----------------+----------+-----------------+
|Date_reported|Country_code|Country|WHO_region|New_cases|Cumulative_cases|New_deaths|Cumulative_deaths|
+-------------+------------+-------+----------+---------+----------------+----------+-----------------+
|   2022-02-05|          BR| Brazil|      AMRO|   298408|        26091520|      1041|           630001|
+-------------+------------+-------+----------+---------+----------------+----------+-----------------+
only showing top 1 row

Wall time: 282 ms


In [116]:
%%time
#Quantos casos foram reportados nesse dia?
consulta3 = (covid
             .filter((covid.Country == 'Brazil')&(covid.New_cases >0))
             .sort(col("New_cases").desc())
             .select('New_cases')
            )
consulta3.show(1)

+---------+
|New_cases|
+---------+
|   298408|
+---------+
only showing top 1 row

Wall time: 247 ms


In [117]:
%%time
consulta4 = (covid.filter( (covid.New_cases > 0 ))
                          .groupby('Date_reported')
                          .agg( f.sum('New_Cases' ).alias('Total') )
                          .where( covid.Date_reported < '2020-02-01' )
                          .sort('Date_reported')
                         
            )
consulta4.show()

+-------------+-----+
|Date_reported|Total|
+-------------+-----+
|   2020-01-04|    3|
|   2020-01-06|    3|
|   2020-01-11|    1|
|   2020-01-12|   42|
|   2020-01-13|    1|
|   2020-01-14|    5|
|   2020-01-17|    4|
|   2020-01-18|    4|
|   2020-01-19|   77|
|   2020-01-20|   79|
|   2020-01-21|   93|
|   2020-01-22|  148|
|   2020-01-23|  134|
|   2020-01-24|  275|
|   2020-01-25|  468|
|   2020-01-26|  704|
|   2020-01-27|  790|
|   2020-01-28| 1779|
|   2020-01-29| 1491|
|   2020-01-30| 1756|
+-------------+-----+
only showing top 20 rows

Wall time: 901 ms


In [123]:
%%time
#Qual país apresentou o maior número de casos em Janeiro de 2022?
consulta5 = (covid           
             .filter((covid.Date_reported >= '2022-01-01')&(covid.Date_reported <= '2022-01-31'))             
             .groupby('Country')
             .agg( f.sum('New_Cases' ).alias('Total') )
             .sort(col("Total").desc())
            )
consulta5.show()

+--------------------+--------+
|             Country|   Total|
+--------------------+--------+
|United States of ...|20378558|
|              France| 9069357|
|               India| 6463636|
|               Italy| 4944057|
|  The United Kingdom| 3983563|
|               Spain| 3643565|
|              Brazil| 2950788|
|             Germany| 2788614|
|           Argentina| 2672438|
|              Turkey| 2084857|
|           Australia| 1789613|
|              Israel| 1474569|
|  Russian Federation| 1361095|
|              Mexico| 1306158|
|            Portugal| 1253029|
|         Netherlands| 1195282|
|             Belgium| 1050642|
|               Japan|  936313|
|              Canada|  929074|
|                Peru|  908994|
+--------------------+--------+
only showing top 20 rows

Wall time: 1.33 s


In [125]:
%%time
#Qual foi o maior número de casos já reportado?
consulta6 = (covid.select('New_cases').sort(col("New_Cases").desc()))
consulta6.show(1)    

+---------+
|New_cases|
+---------+
|  1252717|
+---------+
only showing top 1 row

Wall time: 278 ms


In [151]:
#Qual é a média de casos reportados por dia?
consulta7 = (covid.groupby('Date_reported').agg(f.mean('New_cases').alias('media'))
            
            )
consulta7.agg(f.sum('media')).show()

consulta7.show()   
            

+-----------------+
|       sum(media)|
+-----------------+
|2141508.185045628|
+-----------------+

+-------------+------------------+
|Date_reported|             media|
+-------------+------------------+
|   2020-08-24|1102.9831223628692|
|   2021-01-27|2159.8354430379745|
|   2021-06-22|1297.8776371308018|
|   2021-08-27|3026.1772151898736|
|   2021-10-11|1484.0168776371308|
|   2021-11-13|2158.3586497890296|
|   2021-12-18|3116.5485232067513|
|   2022-03-28| 4159.881856540084|
|   2020-01-21|0.3924050632911392|
|   2020-07-24| 1230.253164556962|
|   2020-08-05| 936.7341772151899|
|   2020-11-29| 2482.168776371308|
|   2021-10-02|2045.4177215189873|
|   2021-11-25|2741.7763713080167|
|   2020-04-30|323.57805907172997|
|   2020-08-28| 1231.126582278481|
|   2020-08-29|1204.6160337552742|
|   2022-01-31|11847.350210970464|
|   2020-10-04|  1404.26582278481|
|   2021-04-29|3703.2236286919833|
+-------------+------------------+
only showing top 20 rows



In [148]:
%%time
#Qual país apresentou o maior número de casos?
consulta8 = (covid           
             
             .groupby('Country')
             .agg( f.sum('New_Cases' ).alias('Total') )
             .sort(col("Total").desc())
            )
consulta8.show()

+--------------------+--------+
|             Country|   Total|
+--------------------+--------+
|United States of ...|80174978|
|               India|43060086|
|              Brazil|30345654|
|              France|27497092|
|             Germany|24200596|
|  The United Kingdom|21933210|
|  Russian Federation|18144788|
|   Republic of Korea|16929564|
|               Italy|16136057|
|              Turkey|15018547|
|               Spain|11786036|
|            Viet Nam|10563502|
|           Argentina| 9072230|
|         Netherlands| 8035603|
|               Japan| 7660012|
|Iran (Islamic Rep...| 7212395|
|            Colombia| 6091094|
|           Indonesia| 6044150|
|              Poland| 5991464|
|              Mexico| 5733514|
+--------------------+--------+
only showing top 20 rows

Wall time: 759 ms
